```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**
---

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_DECEASED_DAILY_PER_REGION]'))
BEGIN
CREATE TABLE [VWSDEST].[RIVM_DECEASED_DAILY_PER_REGION](
	[ID] [INT] PRIMARY KEY IDENTITY(1,1),
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[DECEASED_ACTUAL] [int] NULL,
	[DECEASED_CUMULATIVE] [int] NULL,
	[VRCODE] [varchar](5) NULL,
	[DATE_LAST_INSERTED] [datetime] NULL DEFAULT GETDATE(),
	[WEEK_START] [datetime] NULL,
	[DECEASED_7D_AVG] [decimal](16, 2) NULL,
	[WEEK_START_LAG] [datetime] NULL,
	[DECEASED_7D_AVG_LAG] [decimal](16, 2) NULL
)
GO

END


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY]'))
BEGIN
CREATE TABLE [VWSDEST].[RIVM_DECEASED_DAILY_PER_MUNICIPALITY](
	[ID] [INT] PRIMARY KEY IDENTITY(1,1),
	[DATE_OF_REPORT] [datetime] NULL,
	[DATE_OF_REPORT_UNIX] [bigint] NULL,
	[MUNICIPALITY_CODE] [varchar](10) NULL,
	[OLD_DATE_OF_REPORT] [datetime] NULL,
	[OLD_DATE_OF_REPORT_UNIX] [bigint] NULL,
	[OLD_VALUE] [int] NULL,
	[DIFFERENCE] [int] NULL,
	[DECEASED_ACTUAL] [int] NULL,
	[DECEASED_CUMULATIVE] [int] NULL,
	[DATE_LAST_INSERTED] [datetime] NULL DEFAULT GETDATE(),
	[WEEK_START] [datetime] NULL,
	[DECEASED_7D_AVG] [decimal](16, 2) NULL,
	[WEEK_START_LAG] [datetime] NULL,
	[DECEASED_7D_AVG_LAG] [decimal](16, 2) NULL
)
GO

END


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[RIVM_DECEASED_PER_WEEK]'))
BEGIN
CREATE TABLE [VWSDEST].[RIVM_DECEASED_PER_WEEK](
	[ID] [INT] PRIMARY KEY IDENTITY(1,1),
	[WEEK_START] [datetime] NULL,
	[WEEK_END] [datetime] NULL,
	[WEEK_START_UNIX] [bigint] NULL,
	[WEEK_END_UNIX] [bigint] NULL,
	[DECEASED_ACTUAL] [int] NULL,
	[VRCODE] [varchar](5) NULL,
	[DATE_LAST_INSERTED] [datetime] NULL DEFAULT GETDATE()
)
GO

END


# **VIEWS**
---

### V_RIVM_DECEASED_DAILY_NATIONAL

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_DAILY_NATIONAL]
AS
--Retrieve the totals deceased on NL level by summing the region numbers.
WITH BASE_CTE AS (
        SELECT
        SUM(DECEASED_ACTUAL) AS DECEASED_ACTUAL
    ,   DATE_OF_REPORT_UNIX
    , DATE_LAST_INSERTED
    FROM VWSDEST.RIVM_DECEASED_DAILY_PER_REGION
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.RIVM_DECEASED_DAILY_PER_REGION)
    GROUP BY DATE_OF_REPORT_UNIX, DATE_LAST_INSERTED
)
,
--Determine the cumulative number as COVID_TOTAL
SECOND_CTE AS
(
    SELECT
        a.DECEASED_ACTUAL AS COVID_DAILY
    ,   SUM(DECEASED_ACTUAL) OVER(ORDER BY DATE_OF_REPORT_UNIX ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS COVID_TOTAL
    ,   DATE_OF_REPORT_UNIX AS DATE_UNIX
    ,   [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX

    -- 7d average
	,LAG ([DATE_OF_REPORT_UNIX] ,6) OVER (ORDER BY [DATE_OF_REPORT_UNIX] ASC) WEEK_START
	,LAG ([DATE_OF_REPORT_UNIX] ,7) OVER (ORDER BY [DATE_OF_REPORT_UNIX] ASC) WEEK_START_LAG
	,AVG(CAST([DECEASED_ACTUAL] AS FLOAT)) OVER (ORDER BY DATE_OF_REPORT_UNIX ASC ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as DECEASED_7D_AVG
	,AVG(CAST([DECEASED_ACTUAL] AS FLOAT)) OVER (ORDER BY DATE_OF_REPORT_UNIX ASC ROWS BETWEEN 7 PRECEDING AND 1 PRECEDING) as DECEASED_7D_AVG_LAG
	
    FROM BASE_CTE a
)

SELECT
    COVID_DAILY
,   IIF(WEEK_START IS NULL, NULL, ROUND(DECEASED_7D_AVG,1)) AS COVID_DAILY_MOVING_AVERAGE
,   COVID_TOTAL
,   DATE_UNIX
,   DATE_OF_INSERTION_UNIX
FROM SECOND_CTE
WHERE dbo.CONVERT_UNIX_TO_DATETIME(DATE_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
GO




### V_RIVM_DECEASED_DAILY_PER_MUNICIPALITY

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_DAILY_PER_MUNICIPALITY]
AS

SELECT
    DECEASED_ACTUAL AS COVID_DAILY
,   DECEASED_7D_AVG AS COVID_DAILY_MOVING_AVERAGE
,	DECEASED_CUMULATIVE AS COVID_TOTAL
,	MUNICIPALITY_CODE AS GMCODE
,   DATE_OF_REPORT_UNIX AS DATE_UNIX
,   [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.RIVM_DECEASED_DAILY_PER_MUNICIPALITY
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.RIVM_DECEASED_DAILY_PER_MUNICIPALITY)
AND dbo.CONVERT_UNIX_TO_DATETIME(DATE_OF_REPORT_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
GO




### V_RIVM_DECEASED_DAILY_REGIONAL

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_DAILY_REGIONAL]
AS
SELECT
    DECEASED_ACTUAL AS COVID_DAILY
,   DECEASED_7D_AVG AS COVID_DAILY_MOVING_AVERAGE
,   DECEASED_CUMULATIVE AS COVID_TOTAL
,   VRCODE
,   DATE_OF_REPORT_UNIX AS DATE_UNIX
,   DBO.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM VWSDEST.RIVM_DECEASED_DAILY_PER_REGION
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.RIVM_DECEASED_DAILY_PER_REGION)
AND dbo.CONVERT_UNIX_TO_DATETIME(DATE_OF_REPORT_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
GO




### V_RIVM_DECEASED_PER_WEEK

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_PER_WEEK]
AS
WITH BASE_CTE AS (
    SELECT
    SUM(DECEASED_ACTUAL) AS DECEASED_ACTUAL
,   WEEK_START_UNIX AS WEEK_START_UNIX
,   MAX(DATE_LAST_INSERTED) AS DATE_LAST_INSERTED
FROM [VWSDEST].[RIVM_DECEASED_PER_WEEK]
WHERE WEEK_START >= '2020-03-16 00:00:00.000'
AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[RIVM_DECEASED_PER_WEEK])
GROUP BY WEEK_START_UNIX
)

SELECT
    DECEASED_ACTUAL AS COVID_DAILY
,   SUM(DECEASED_ACTUAL) OVER(ORDER BY WEEK_START_UNIX ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS COVID_TOTAL
,   WEEK_START_UNIX AS DATE_UNIX
,   [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM BASE_CTE
WHERE dbo.CONVERT_UNIX_TO_DATETIME(WEEK_START_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
GO




### V_RIVM_DECEASED_PER_WEEK_PER_REGION

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.
CREATE OR ALTER VIEW [VWSDEST].[V_RIVM_DECEASED_PER_WEEK_PER_REGION]
AS
SELECT
    DECEASED_ACTUAL AS COVID_DAILY
,   SUM(DECEASED_ACTUAL) OVER(PARTITION BY VRCODE ORDER BY WEEK_START_UNIX ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS COVID_TOTAL
,   VRCODE
,   WEEK_START_UNIX AS DATE_UNIX
,   [dbo].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSDEST].[RIVM_DECEASED_PER_WEEK]
WHERE WEEK_START >= '2020-03-16 00:00:00.000'
AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[RIVM_DECEASED_PER_WEEK])
AND dbo.CONVERT_UNIX_TO_DATETIME(WEEK_START_UNIX) < '2023-01-01' -- added as RIVM reports 9999 after this date
GO


